In [1]:
import os
import numpy as np
from matplotlib import image
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

In [2]:
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

print(tensorflow.keras.__version__)

2.2.4-tf


In [3]:
#define parameters for the Convolutional Neural Network
image_size = 128
depth = 3 #3 for RGB, 1 for Grayscale
num_categories = 9
num_epochs = 30
batch_size = 32
learning_rate = 0.0005

#load images from PlantVillage folder
data_directory = "C:/Users/Reuben/Desktop/Computer Vision/Datasets/Tomato Leaf Raw"
categories = []
folder_list = os.scandir(data_directory)
for folder in folder_list:
    categories.append(folder.name)
    
data = []
labels = []

for category in categories:
    path = os.path.join(data_directory, category)
    
    class_num = categories.index(category)
    print(class_num, category)
        
    for img in os.listdir(path)[:950]:
        img = tensorflow.keras.preprocessing.image.load_img(os.path.join(path, img), target_size=(128, 128))
        new_img = tensorflow.keras.preprocessing.image.img_to_array(img)
        #input images into NumPy arrays Data and Label
        data.append(new_img)
        labels.append(class_num)
        
data = np.array(data, dtype=np.float64)/255.0
labels = np.array(labels)

labels = np.eye(num_categories)[labels] 

data, labels = shuffle(data, labels, random_state=0)
print(data.shape)

0 Tomato___Bacterial_spot
1 Tomato___Early_blight
2 Tomato___healthy
3 Tomato___Late_blight
4 Tomato___Leaf_Mold
5 Tomato___Septoria_leaf_spot
6 Tomato___Spider_mites Two-spotted_spider_mite
7 Tomato___Target_Spot
8 Tomato___Tomato_Yellow_Leaf_Curl_Virus
(8550, 128, 128, 3)


In [4]:
train_data = data[0:5985]
train_labels = labels[0:5985]

test_data = data[5985:8550]
test_labels = labels[5985:8550]

In [5]:
datagen = ImageDataGenerator(
    rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, 
    zoom_range=0.2,horizontal_flip=True, 
    fill_mode="nearest")

In [6]:
model = Sequential()
inputShape = (image_size, image_size, depth)
chanDim = -1
if K.image_data_format() == "channels_first":
    inputShape = (depth, height, width)
    chanDim = 1
    
model.add(Conv2D(32, (3, 3), padding="same", activation='relu', input_shape=inputShape))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding="same", activation='relu'))
model.add(BatchNormalization(axis=chanDim))

model.add(Conv2D(64, (3, 3), padding="same", activation='relu'))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding="same", activation='relu'))
model.add(BatchNormalization(axis=chanDim))

model.add(Conv2D(128, (3, 3), padding="same", activation='relu'))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(num_categories))
model.add(Activation("softmax"))


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [7]:
opt = Adam(lr=learning_rate, decay=learning_rate / num_epochs)
# distribution
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
# train the network
print("[INFO] training network...")
model.summary()

[INFO] training network...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 128, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 42, 42, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 42, 42, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 42, 42, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 42, 42, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            

In [8]:
history = model.fit_generator(
    datagen.flow(train_data, train_labels, batch_size=batch_size),
    validation_data=(test_data, test_labels),
    steps_per_epoch=len(train_data) // batch_size,
    epochs=num_epochs, verbose=1
    )

Epoch 1/30
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
187/187 [==============================] - 47s 249ms/step - loss: 0.2441 - acc: 0.9121 - val_loss: 1.4035 - val_acc: 0.8017
Epoch 2/30
187/187 [==============================] - 40s 215ms/step - loss: 0.1629 - acc: 0.9393 - val_loss: 1.0898 - val_acc: 0.8220
Epoch 3/30
187/187 [==============================] - 40s 215ms/step - loss: 0.1445 - acc: 0.9475 - val_loss: 0.5707 - val_acc: 0.8581
Epoch 4/30
187/187 [==============================] - 40s 215ms/step - loss: 0.1164 - acc: 0.9566 - val_loss: 0.5482 - val_acc: 0.8880
Epoch 5/30
187/187 [==============================] - 40s 215ms/step - loss: 0.1039 - acc: 0.9629 - val_loss: 1.3013 - val_acc: 0.8284
Epoch 6/30
187/187 [==============================] - 40s 214ms/step - loss: 0.1093 - acc: 0.9596 - val_loss: 0.1691 - val_acc: 0.9503
Epoch 7/30
187/187 [==============================] - 40s 215ms/step - loss: 0.0882 - acc: 0.966

187/187 [==============================] - 27s 144ms/step - loss: 0.0398 - acc: 0.9857 - val_loss: 0.5347 - val_acc: 0.9192
Epoch 24/30
187/187 [==============================] - 25s 132ms/step - loss: 0.0382 - acc: 0.9861 - val_loss: 0.4761 - val_acc: 0.9271
Epoch 25/30
187/187 [==============================] - 23s 125ms/step - loss: 0.0354 - acc: 0.9882 - val_loss: 0.1018 - val_acc: 0.9701
Epoch 26/30
187/187 [==============================] - 23s 124ms/step - loss: 0.0407 - acc: 0.9852 - val_loss: 0.5180 - val_acc: 0.9202
Epoch 27/30
187/187 [==============================] - 27s 145ms/step - loss: 0.0347 - acc: 0.9878 - val_loss: 0.5011 - val_acc: 0.9241
Epoch 28/30
187/187 [==============================] - 38s 204ms/step - loss: 0.0319 - acc: 0.9886 - val_loss: 0.1673 - val_acc: 0.9653
Epoch 29/30
187/187 [==============================] - 36s 193ms/step - loss: 0.0328 - acc: 0.9884 - val_loss: 0.0991 - val_acc: 0.9668
Epoch 30/30
187/187 [==============================] - 37s 1

In [9]:
model.save('C:/Users/Reuben/Desktop/Computer Vision/TF Scale/Leaf Detection/Models/Tomato Leaf Classification.h5')

In [ ]:
import tensorflowjs
tfjs.converters.save_keras_model(model, "C:/Users/Reuben/Desktop/Computer Vision/TF Scale/Leaf Detection/Models")